In [62]:
import pandas as pd
import os
os.chdir("F://NMIMS//sem2//ML//DATASETS FOR ML//heart")
df=pd.read_csv("heart.csv")
df.head()
import warnings 
warnings.filterwarnings("ignore")


In [35]:
df.isna().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [36]:
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
target        int64
dtype: object

In [37]:
#by normalising the numerical fields here X6 is categorical
X6=df.drop(["age","trestbps","chol","thalach","oldpeak"],1)
X5=df[["trestbps","chol","thalach","oldpeak","age"]]
X5.head()
#X6.head()

,trestbps,chol,thalach,oldpeak,age
0,145,233,150,2.3,63
1,130,250,187,3.5,37
2,130,204,172,1.4,41
3,120,236,178,0.8,56
4,120,354,163,0.6,57


In [38]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
cols=list(X5) 
X5=scaler.fit_transform(X5)
X5=pd.DataFrame(X5,columns=cols)
X5.head()


C:\Users\sanat\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,trestbps,chol,thalach,oldpeak,age
0,0.481132,0.244292,0.603053,0.370968,0.708333
1,0.339623,0.283105,0.885496,0.564516,0.166667
2,0.339623,0.178082,0.770992,0.225806,0.250000
3,0.245283,0.251142,0.816794,0.129032,0.562500
4,0.245283,0.520548,0.702290,0.096774,0.583333


In [39]:
df=pd.concat([X5,X6],1)
df.head()

,trestbps,chol,thalach,oldpeak,age,sex,cp,fbs,restecg,exang,slope,ca,thal,target
0,0.481132,0.244292,0.603053,0.370968,0.708333,1,3,1,0,0,0,0,1,1
1,0.339623,0.283105,0.885496,0.564516,0.166667,1,2,0,1,0,0,0,2,1
2,0.339623,0.178082,0.770992,0.225806,0.250000,0,1,0,0,0,2,0,2,1
3,0.245283,0.251142,0.816794,0.129032,0.562500,1,1,0,1,0,2,0,2,1
4,0.245283,0.520548,0.702290,0.096774,0.583333,0,0,0,1,1,2,0,2,1


In [40]:
X=df.drop(["ca"],1)
y=df["ca"]
y.head()
X.head()


,trestbps,chol,thalach,oldpeak,age,sex,cp,fbs,restecg,exang,slope,thal,target
0,0.481132,0.244292,0.603053,0.370968,0.708333,1,3,1,0,0,0,1,1
1,0.339623,0.283105,0.885496,0.564516,0.166667,1,2,0,1,0,0,2,1
2,0.339623,0.178082,0.770992,0.225806,0.250000,0,1,0,0,0,2,2,1
3,0.245283,0.251142,0.816794,0.129032,0.562500,1,1,0,1,0,2,2,1
4,0.245283,0.520548,0.702290,0.096774,0.583333,0,0,0,1,1,2,2,1


In [52]:
#backward elimination for feature extraction
import statsmodels.api as sm
cols = list(X)
p_max = 1
while (len(cols)>0):
    p=[]
    X_1= X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = list(model.pvalues.values[1:])
    p = pd.Series(p,index = cols)
    p_max = max(p)
    features_with_p_max = p.idxmax()
    if (p_max>0.05):
        cols.remove(features_with_p_max)
    else:
        break
selected_features = cols
selected_features

['age', 'fbs', 'target']

# Logistic Reg

In [53]:
from sklearn.model_selection import train_test_split as tts
X_train,X_test,y_train,y_test=tts(X,y,test_size=0.2,random_state=42)
#logistics
from sklearn.metrics import r2_score,mean_squared_error,median_absolute_error
from sklearn import metrics
from sklearn.linear_model import LogisticRegression,RidgeCV,LassoCV,ElasticNetCV

log_reg=LogisticRegression(multi_class="multinomial",solver='newton-cg').fit(X_train,y_train)
y_pred= log_reg.predict(X_test)
metrics.accuracy_score(y_pred,y_test)
#metrics.confusion_matrix(y_pred,y_test)
#y_pred

0.5737704918032787

In [83]:
# Logistic Regression with Crossvalscore
cross_val = cross_val_score(log_reg,X_train,y_train,cv=5)
print('Average Cross Validation Score for Linear Regression : ',cross_val.mean()) # cross verification

Average Cross Validation Score for Linear Regression :  0.6162802216168614


In [54]:
#with features
filter2=X[selected_features]
from sklearn.model_selection import train_test_split as tts
filter2_train,filter2_test,y_train,y_test=tts(filter2,y,test_size=0.2,random_state=42)
#logistics
from sklearn.metrics import r2_score,mean_squared_error,median_absolute_error
from sklearn import metrics
from sklearn.linear_model import LogisticRegression,RidgeCV,LassoCV,ElasticNetCV
log_reg=LogisticRegression(multi_class="multinomial",solver='newton-cg').fit(filter2_train,y_train)
y_pred= log_reg.predict(filter2_test)
metrics.accuracy_score(y_pred,y_test)
#metrics.confusion_matrix(y_pred,y_test)

0.47540983606557374

# Randomforest

#----------->random forest
from sklearn.ensemble import RandomForestClassifier

regressor = RandomForestClassifier(n_estimators=20, random_state=0)  
regressor.fit(X_train, y_train)  
y_pred = regressor.predict(X_test) 
metrics.accuracy_score(y_pred,y_test)

In [64]:
#-----------> Random Forest with GRID SEARCH
rfc = RandomForestClassifier()
from sklearn.model_selection import GridSearchCV

param_grid = [{"n_estimators":[3,10,30], "max_features":[2,4]} #max features not greater than no.of features in dataset
              ,{"bootstrap": [False],"n_estimators":[3,10],"max_features":[2,3,4]},]
rfc = GridSearchCV(rfc,param_grid,cv=5,scoring="accuracy")
rfc.fit(X_train,y_train)
y_pred = rfc.predict(X_test)
metrics.accuracy_score(y_pred,y_test)

0.5081967213114754

In [63]:
#-----------> Random Forest with CROSS VALIDATION
from sklearn.model_selection import cross_val_score
cross_val_score(rfc, X, y, cv = 5).mean()

0.5606010928961748

In [45]:
#------------>random forest with features
from sklearn.ensemble import RandomForestClassifier
regressor = RandomForestClassifier(n_estimators=20, random_state=0)  
regressor.fit(filter2_train, y_train)  
y_pred = regressor.predict(filter2_test) 
metrics.accuracy_score(y_pred,y_test)

0.5901639344262295

# KNN

In [49]:
#---------->knn
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=7)
#KNN
knn.fit(X_train,y_train)
y_pred=knn.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))
metrics.confusion_matrix(y_pred,y_test)
metrics.accuracy_score(y_pred,y_test)

0.5245901639344263

In [65]:
#--------->KNN with Crossvalscore
cross_val_score(knn, X, y, cv = 10).mean() # Cross validating the model on 10-folds

0.5797105911330049

In [81]:
#--------->KNN with Grid Search CV

params = {"n_neighbors":np.arange(1,16)}
params
knn = KNeighborsClassifier()
knn_gs = GridSearchCV(knn,param_grid=params, cv=10, scoring="accuracy")
knn_gs.fit(X,y)
knn_gs.best_score_

0.5874587458745875

In [50]:
#knn with features
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=7)
#KNN
knn.fit(filter2_train,y_train)
y_pred=knn.predict(filter2_test)
np.sqrt(mean_squared_error(y_test,y_pred))
metrics.confusion_matrix(y_pred,y_test)
metrics.accuracy_score(y_pred,y_test)

0.5081967213114754